<p style="text-align: center; font-size: 300%"> Computational Finance </p>
<img src="img/ABSlogo.svg" alt="LOGO" style="display:block; margin-left: auto; margin-right: auto; width: 50%;">

# Dealing with Data
## Working with `datetime` objects.


## The `pandas` library
### Dataframes


### Fetching Data
* `pandas_datareader` is a package that makes it easy to fetch financial data from the web.
* It used to be included in pandas (and therefore Anaconda). In newer versions, you'll have to do `conda install pandas-datareader` to install it
* See http://pandas-datareader.readthedocs.io/en/latest/remote_data.html for the documentation.

In [18]:
#!conda install -y pandas-datareader #uncomment to install. (Note ! executes shell commands)
import pandas_datareader.data as web

In [38]:
start = pd.datetime(2010, 1, 1)
end = pd.datetime(2013, 1, 27)
f = web.DataReader("AAPL", 'yahoo', start, end)


## Plotting with `matplotlib`